<a href="https://colab.research.google.com/github/gugi200/final_project/blob/main/scikit_models_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install wandb -qU
!wandb login 3014974e724f01c4d63f956fa13fd7f0463e16d4
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 11.9 MB/s eta 0:00:00


In [30]:
# processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ML libraries
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


from sklearn.decomposition import PCA

from sklearn.pipeline import make_pipeline

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# torch functionality functions
import torch

# for downloading the data
from pathlib import Path
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torchvision import datasets
from sklearn.utils import Bunch

# helper functions
import wandb
from timeit import default_timer as timer
from tqdm.auto import tqdm

# download data for testing

In [6]:
# download the test data

from torchvision.datasets import FashionMNIST
from torchvision import transforms
train_data = FashionMNIST(
    root='~/.pytorch/F_MNIST_data',
    download=True,
    train=True,
    target_transform=None)

test_data = FashionMNIST(
    root='~/.pytorch/F_MNIST_data',
    download=True,
    train=False,
    target_transform=None)

class_names = [x.replace('/', '_').replace('-', '') for x in train_data.classes]

transform = transforms.ToPILImage()



from pathlib import Path

data_path = Path("data/")
image_path1 = data_path / "FashionMNIST" / "train"

if image_path1.is_dir():
    print('directory already exists')
else:
    print('creating folder')
    for class_name in class_names:
        image_class_path = image_path1 / class_name
        image_class_path.mkdir(parents=True, exist_ok=True)

    for idx, (train_img, target) in enumerate(zip(train_data.data[:3000], train_data.targets[:3000])):
        train_data_PIL = transform(train_img)
        file = "data_" + str(idx) + '.jpg'
        # print(target)
        # print(data_path / "FashionMNIST"/"train" / class_names[target] /file)
        train_data_PIL.save(data_path / "FashionMNIST"/"train" / class_names[target] /file)


data_path = Path("data/")
image_path2 = data_path / "FashionMNIST" / "test"
if image_path2.is_dir():
    print('directory already exists')
else:
    print('creating folder')
    for class_name in class_names:
        image_class_path = image_path2 / class_name
        image_class_path.mkdir(parents=True, exist_ok=True)
        image_class_path.mkdir(parents=True, exist_ok=True)

    for idx, (test_img, target) in enumerate(zip(test_data.data[:250], test_data.targets[:250])):
        test_data_PIL = transform(test_img)
        file = "data_" + str(idx) + '.jpg'
        test_data_PIL.save(data_path / "FashionMNIST"/"test" / class_names[target] /file)





dataset = datasets.ImageFolder(root=Path("data/FashionMNIST/train"),
                                  transform=np.array,
                                  target_transform=None) # transform for label



class_names = dataset.classes

def torch_dataset_to_numpy(dataset, target_names):
    for i, (data, target) in enumerate(dataset):

        if i==0:
            data_numpy = data
            data_numpy = np.expand_dims(data_numpy, axis=0)
            target_numpy = np.array([target])
            target_numpy = np.expand_dims(target_numpy, axis=0)
        else:
            data = np.expand_dims(data, axis=0)
            target = np.expand_dims(np.array([target]), axis=0)
            data_numpy = np.append(data_numpy, data, axis=0)
            target_numpy = np.append(target_numpy, target, axis=0)

    return Bunch(data=data_numpy, target=target_numpy, target_names=target_names)


custom_dataset = torch_dataset_to_numpy(dataset, class_names)



100%|██████████| 26421880/26421880 [00:01<00:00, 17607036.14it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 298688.14it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5328602.61it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 10641831.93it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw

creating folder
creating folder


In [33]:
from sklearn.datasets import load_iris
iris = load_iris()
iris.data.shape, iris.target.shape

((150, 4), (150,))

# Data preprocessing

In [ ]:
# normalize the data
def normalize(data, mode):
    ''''''


def visualizeData(data):




In [41]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(custom_dataset.data,
                                                    custom_dataset.target,
                                                    test_size=0.25,
                                                    random_state=42,
                                                    shuffle=True)


X_train, X_test = X_train[:, :, :, 0].reshape((2250, 784)), X_test[:, :, :, 0].reshape((750, 784))




X_train.shape, y_train.shape






((2250, 784), (2250, 1))

# create pipelines

In [ ]:

param_RF = {
'max_depth': [2, 3, 4, 5, 6, 7, 8],
'n_estimators': [50, 75, 100, 125, 150]
}

grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_RF, cv=5, n_jobs=-1)
grid_rf.fit(X_train, y_train)
print('random forest')
print('results: ', grid_rf.cv_results_)
print('Best estimator: ', grid_rf.best_estimator_)
print('Best params: ', grid_rf.best_params_)
print('Best score: ', grid_rf.best_score_)


In [44]:
param_RF = {
'max_depth': [2, 3, 4, 5, 6, 7, 8],
'n_estimators': [50, 75, 100, 125, 150]
}

grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_RF, cv=5, n_jobs=-1)
grid_rf.fit(X_train, y_train)
print('random forest')
print('results: ', grid_rf.cv_results_)
print('Best estimator: ', grid_rf.best_estimator_)
print('Best params: ', grid_rf.best_params_)
print('Best score: ', grid_rf.best_score_)



param_DT = {
'max_depth': [2, 3, 4, 5, 6, 7, 8]
}

grid_DT = GridSearchCV(DecisionTreeClassifier(random_state=42), param_DT, cv=5, n_jobs=-1)
grid_DT.fit(X_train, y_train)
print('decisionb forest')
print('results: ', grid_DT.cv_results_)
print('Best estimator: ', grid_DT.best_estimator_)
print('Best params: ', grid_DT.best_params_)
print('Best score: ', grid_DT.best_score_)




param_svc = {
'max_depth': [2, 3, 4, 5, 6, 7, 8],
'n_estimators': [50, 75, 100, 125, 150]
}

grid_svc_std = GridSearchCV(RandomForestClassifier(random_state=42), param_svc, cv=5, n_jobs=-1)
stdScaled = StandardScaler()
scaled_X_train = stdScaled.fit(X_train)
scaled_X_test = stdScaled.fit(X_test)
grid_svc_std.fit(scaled_X_train, y_train)
print('random forest')
print('results: ', grid_svc_std.cv_results_)
print('Best estimator: ', grid_svc_std.best_estimator_)
print('Best params: ', grid_svc_std.best_params_)
print('Best score: ', grid_svc_std.best_score_)



param_grid_svc = {
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
    }


grid_svc_mm = GridSearchCV(SVC(), param_grid_svc, cv=5, n_jobs=-1)
mmScaled = MinMaxScaler()
scaled_X_train = mmScaled.fit(X_train)
grid_svc_mm.fit(X_train, y_train)
print('svc minmax')
print('results: ', grid_svc_mm.cv_results_)
print('Best estimator: ', grid_svc_mm.best_estimator_)
print('Best params: ', grid_svc_mm.best_params_)
print('Best score: ', grid_svc_mm.best_score_)






/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


random forest
results:  {'mean_fit_time': array([1.02981625, 0.69544811, 0.93445668, 1.1324718 , 1.34784713,
       1.08775377, 1.29632831, 1.22485542, 1.49163642, 2.45816541,
       1.11098719, 1.18292022, 1.52057009, 2.6504724 , 2.60322495,
       0.92594523, 1.68441567, 2.56732011, 2.35032148, 4.08330398,
       1.0748188 , 1.58677959, 3.08319945, 2.59273248, 4.09134698,
       1.17374802, 1.90963349, 3.15515871, 3.20076413, 4.36513448,
       1.32240148, 2.87189927, 2.62748241, 4.192308  , 4.18823419]), 'std_fit_time': array([0.47641719, 0.06371576, 0.01325129, 0.04078968, 0.11474857,
       0.11461023, 0.28901035, 0.02535643, 0.05016825, 0.57695385,
       0.20725074, 0.01431653, 0.01337977, 0.66014715, 0.43666023,
       0.01894117, 0.38506402, 0.63521165, 0.03546187, 0.86280688,
       0.01545269, 0.02315913, 0.55686731, 0.02814028, 0.94936584,
       0.00790228, 0.29148317, 0.71906346, 0.40116181, 0.90718303,
       0.03349304, 0.43427577, 0.08491052, 0.87220251, 0.36607447]), 

TypeError: ignored